In [10]:
# calcul matriciel
import numpy as np
# utilisation des modèles
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score

In [11]:
class MNISTLoader:
  def __init__(self):
    self.x_train = None
    self.x_test = None
    self.x_val = None
    
    self.x_test_normalized = None
    self.x_train_normalized_80 = None
    self.x_val_normalized_20 = None
    self.y_test = None
    self.y_train_80 = None
    self.y_val_20 = None


  def load_data(self):
    (x_train, y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data()
    x_train_normalized = x_train / 255
    self.x_test_normalized = self.x_test / 255

    self.x_train, self.x_val, self.y_train_80, self.y_val_20 = train_test_split(x_train, y_train, test_size=0.2, random_state=42)
    self.x_train_normalized_80, self.x_val_normalized_20, self.y_train_80, self.y_val_20 = train_test_split(x_train_normalized, y_train, test_size=0.2, random_state=42)


loader = MNISTLoader()
loader.load_data()


# Convolutional model with handly data

In [12]:
num_classes = 10
core_size = 4
model = Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(128, core_size, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, core_size, activation='relu'),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(loader.x_train_normalized_80,
          loader.y_train_80,
          validation_data=(loader.x_val_normalized_20, loader.y_val_20),
          epochs=)

model.evaluate(loader.x_test_normalized, loader.y_test)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.8963 - loss: 0.3332 - val_accuracy: 0.9821 - val_loss: 0.0580
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9846 - loss: 0.0487 - val_accuracy: 0.9875 - val_loss: 0.0388
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9904 - loss: 0.0300 - val_accuracy: 0.9883 - val_loss: 0.0379
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9931 - loss: 0.0218 - val_accuracy: 0.9873 - val_loss: 0.0419
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9947 - loss: 0.0169 - val_accuracy: 0.9861 - val_loss: 0.0492
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9950 - loss: 0.0150 - val_accuracy: 0.9900 - val_loss: 0.0400
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9961 - loss: 0.0113 - val_accuracy: 0.9897 - val_loss: 0.0439
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.9969 - loss: 0

[0.026035653427243233, 0.9930999875068665]

In [14]:
import tkinter as tk
from PIL import Image, ImageDraw

# Création de la fenêtre principale
window = tk.Tk()

canvas = tk.Canvas(window, width=280, height=280, bg='white')
canvas.pack()

image = Image.new('L', (280, 280), 255)
draw = ImageDraw.Draw(image)

def clear_canvas():
    canvas.delete('all')
    global image, draw
    image = Image.new('L', (280, 280), 255)
    draw = ImageDraw.Draw(image)

clear_button = tk.Button(window, text='Clear', command=clear_canvas)
clear_button.pack()


# Fonction appelée lors du dessin
def draw_on_canvas(event):
    x, y = event.x, event.y
    canvas.create_oval(x, y, x+10, y+10, fill='black')
    draw.rectangle([x, y, x+10, y+10], fill='black')

# Fonction appelée lors de la prédiction
def predict_digit():
    # Redimensionner l'image à la taille attendue par le modèle (28x28)
    resized_image = image.resize((28, 28))

    # Prétraitement de l'image pour l'adapter au modèle (conversion en tableau numpy, normalisation, etc.)
    preprocessed_image = preprocess_image(resized_image)

    # Effectuer la prédiction avec le modèle
    predictions = model.predict(np.expand_dims([preprocessed_image], axis=-1))[0]
    prediction = np.argmax(predictions)

    # Afficher la prédiction
    prediction_label.config(text='Prediction: ' + str(prediction))

# Fonction pour prétraiter l'image avant la prédiction
def preprocess_image(image):
    # Convertir l'image en niveaux de gris
    image = image.convert('L')

    # Convertir l'image en tableau numpy
    image_array = np.array(image)
    
    # on inverse le noir et le blanc, car le dessin s'affiche mieux sur fond blanc mais notre modèle est entrainé sur fond noir
    # en gros sur l'image 255 correspond à du vide alors que pour notre modèle 255 c'est une case dessinée par l'utilisateur
    final_array = 255 - image_array
    return final_array

predict_button = tk.Button(window, text='Predict', command=predict_digit)
predict_button.pack()

prediction_label = tk.Label(window, text='Prediction: ')
prediction_label.pack()

# Capturer les événements de dessin sur la toile
canvas.bind('<B1-Motion>', draw_on_canvas)

# Lancer l'application
window.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
